# RAG Langchain with Unstructured PDF, FAISS (gpu support), Qwen3 LLM & Qwen3 Embeddings
Name      : Willy Santoso<br>
Subject   : RAG Langchain with Unstructured PDF, FAISS (gpu support), Qwen3 LLM & Qwen3 Embeddings
<br><br>
This code is can be run on Google Colab since this is a Jupyter Notebook file (upload this ipynb to Colab).<br>
If using Google Colab, run this code below to verify Nvidia GPU Driver and install the libraries:

In [ ]:
!nvidia-smi

Mon Oct 13 03:56:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### Install PCI utils for Ollama GPU support, libs for Poppler PDF and Tesseract-ocr for image PDF reader

In [ ]:
!sudo apt-get update
!sudo apt-get install -qq -y pciutils
!sudo apt-get install -qq -y libxml2 libxslt1-dev libmagic-dev
!sudo apt-get install -qq -y libnss3 libnss3-dev
!sudo apt-get install -qq -y libcairo2-dev libjpeg-dev libgif-dev
!sudo apt-get install -qq -y cmake libblkid-dev e2fslibs-dev libboost-all-dev libaudit-dev
!sudo apt-get install -qq -y tesseract-ocr
!sudo apt-get install -qq -y libpoppler-dev poppler-utils

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!pip install -q pyngrok

#### Install Ollama with GPU support, Qwen3 LLM, and Qwen3 Embeddings
We will use `Qwen3-8B` and `Qwen3-embedding-8B` for RAG.<br>
Then we also will be using `Qwen2.5-7B` and `Qwen3-embedding-8B` for RAG evaluation (RAGAS).

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve > server.log 2>&1 &
!ollama pull qwen3-embedding:8b
!ollama pull qwen3-embedding:4b
!ollama pull qwen3:8b
!ollama pull qwen2.5:3b

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.






In [ ]:
!ollama --version

ollama version is 0.12.3


In [ ]:
!ollama list

NAME                  ID              SIZE      MODIFIED      
qwen2.5:3b            357c53fb659c    1.9 GB    3 minutes ago    
qwen3:8b              500a1f067a9f    5.2 GB    3 minutes ago    
qwen3-embedding:4b    df5bd2e3c74c    2.5 GB    3 minutes ago    
qwen2.5:7b            845dbda0ea48    4.7 GB    2 hours ago      
qwen3-embedding:8b    64b933495768    4.7 GB    2 hours ago      


#### Install all required pip libraries for RAG
<b>including LangChain, ChromaDB, FAISS (with gpu support), and RAGAS for evaluation.</b>

In [ ]:
!pip install -q numpy
!pip install -q protobuf
!pip install -q chromadb
!pip install -q onnx
!pip install -q onnxruntime onnxruntime-gpu
!pip install -q rapidocr-onnxruntime
!pip install -q transformers accelerate datasets evaluate rouge_score
!pip install -q sentence-transformers
!pip install -q pytesseract
!pip install -U -q pydantic ollama
!pip install -U -q nltk
!pip install -U -q langchain langchain-core langchain-community
!pip install -U -q langchain-chroma langchain-ollama langchain-huggingface langchainhub langserve langsmith
!pip install -U -q langchain-unstructured unstructured-client unstructured "unstructured[all-docs]" python-magic pydantic lxml pypdf pymupdf
!pip install -U -q ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 87.6 MB/s eta 

In [ ]:
!pip install -q opencv-python==4.10.0.84 opencv-python-headless==4.10.0.84 opencv-contrib-python==4.10.0.84
!pip install -q numpy==1.26.4 faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# Please restart the runtime in order to take changes, otherwise PDFLoader will throw numpy error

### Table of Contents
1. Data Preprocessing
2. Retrieval Strategy
3. Model Selection
4. Evaluation Dataset Creation
5. Evaluation
6. Recommendation

### 1. Data Preprocessing
Here we are preparing the required libraries and load the PDF with `UnstructuredPDFLoader`

<b>Import the required libraries</b><br>
- Langchain: main library/tools for RAG
- Chroma: for storing vector databases that generated from Embeddings
- RecursiveCharacterSplitter: use the recursive text splitter for splitting documents
- UnstructurdPDFLoader: for loading unstructured documents like PDFs.
- ChatOllama: use open-source Ollama for Chat LLMs.
- Ollama Embeddings: use open-source Ollama for Embeddings.
- ChatPromptTemplate and PromptTemplate: template for inserting the prompts.
- StrOutputParser: string output parsers.
- RunnablePassthrough: runnable for passing the question inputs from the user.
- MultiQueryRetriever: the main Retrieval method that we want to use.

In [ ]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

<b>Load the document</b><br>
Let's load the paper

In [ ]:
loader = UnstructuredPDFLoader('./1.Jurnal_MBG_keracunan.pdf')
data = loader.load()

MBG Paper source: https://ojs.ruangpublikasi.com/index.php/jpim/article/view/534

Display head of paper

In [ ]:
print(data[0].page_content[:1000])

JPIM: Jurnal Penelitian Ilmiah Multidisipliner Vol. 01, No. 04, Tahun 2025, Hal. 1329-1342, ISSN: 3089-0128 (Online)

Keracunan Program Makan Siang Bergizi Gratis dalam Tinjauan Hukum Kemasyarkatan dan Aspek Negara Berkembang

Kelvin Felix Vivano Emmanulle1, Radhitya Adji Saputra2, Anta Sena Pratama Yuda Wicaksono3, Dhyavant Pramasthya Wicaksana4, Ubaidillah Kamal5

1,2,3,4,5 Universitas Negeri Semarang

Email : kelvinvivano@students.unnes.ac.id1, radhitya271@students.unnes.ac.id2 antasenapratama04@students.unnes.ac.id3, ggdq6771@students.unnes.ac.id4 ubaidillahkamal@mail.unnes.ac.id5

Article Info Article history: Received July 05, 2025 Revised July 15, 2025 Accepted July 25, 2025

Keywords: Legal protection, Free lunch program, Public participation

ABSTRACT This study examines the legal protection of free lunch programs from the perspective of community law in Indonesia. The aim is to analyze legal basis, monitoring mechanisms, public participation, and implementation challenges to 

<b>Splitting document</b>

For Splitting documents, there is two known method from Langchain:
- `CharacterTextSplitter`: This is a simpler method that splits the text based on a specified character, such as spaces or newlines.
- `RecursiveCharacterTextSplitter`: This method is more advanced and versatile. It attempts to split the text using a series of separators in a hierarchical or recursive manner. For example, it might first try to split the text at paragraph breaks (\n\n), and if the resulting chunks are too large, it then tries to split by single newlines (\n), and if necessary, by spaces, and finally by individual characters.

In [ ]:
# Split and chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

<b>Preview document chunks</b>

In [ ]:
# preview chunks
for item in chunks:
  print(item)
  break

page_content='JPIM: Jurnal Penelitian Ilmiah Multidisipliner Vol. 01, No. 04, Tahun 2025, Hal. 1329-1342, ISSN: 3089-0128 (Online)

Keracunan Program Makan Siang Bergizi Gratis dalam Tinjauan Hukum Kemasyarkatan dan Aspek Negara Berkembang

Kelvin Felix Vivano Emmanulle1, Radhitya Adji Saputra2, Anta Sena Pratama Yuda Wicaksono3, Dhyavant Pramasthya Wicaksana4, Ubaidillah Kamal5

1,2,3,4,5 Universitas Negeri Semarang

Email : kelvinvivano@students.unnes.ac.id1, radhitya271@students.unnes.ac.id2 antasenapratama04@students.unnes.ac.id3, ggdq6771@students.unnes.ac.id4 ubaidillahkamal@mail.unnes.ac.id5

Article Info Article history: Received July 05, 2025 Revised July 15, 2025 Accepted July 25, 2025

Keywords: Legal protection, Free lunch program, Public participation' metadata={'source': './1.Jurnal_MBG_keracunan.pdf'}


In [ ]:
chunks[1]

Document(metadata={'source': './1.Jurnal_MBG_keracunan.pdf'}, page_content='ABSTRACT This study examines the legal protection of free lunch programs from the perspective of community law in Indonesia. The aim is to analyze legal basis, monitoring mechanisms, public participation, and implementation challenges to ensure the program operates with accountability, transparency, and fairness. The method employed is a literature review and qualitative analysis of regulations, oversight reports, and case studies related to the implementation of free lunch programs in primary schools. The findings indicate that the program requires a strong legal foundation aligned with Articles 28C and 28H of the 1945 Constitution, which guarantee the right to food and social security. The absence of binding legal instruments creates opportunities for abuses, including corruption in procurement and limited public involvement in program planning. Legal protection must be strengthened through procurement transp

### 2. Retrieval Strategy
For Retrieval Strategy: here we will be using Dense Retrieval (Embedding Vectors) and `MultiQueryRetriever`, it can send multiple queries at one time and one prompt.<br>
<br>
At first, we test the `OllamaEmbeddings` inference with `qwen3-embedding:8b`

In [ ]:
import ollama

def get_embedding(text: str):
    try:
        response = ollama.embeddings(
            model="qwen3-embedding:8b",
            prompt=text
        )
        return response['embedding']
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return None

In [ ]:
sentence = "Ollama memudahkan untuk menjalankan LLM secara lokal."
embedding = get_embedding(sentence)

In [ ]:
if embedding:
    print(f"Embedding untuk: '{sentence}'")
    # Cetak beberapa dimensi pertama untuk singkatnya
    print(f"5 dimensi pertama: {embedding[:5]}")
    print(f"Total dimensi: {len(embedding)}")

Embedding untuk: 'Ollama memudahkan untuk menjalankan LLM secara lokal.'
5 dimensi pertama: [0.01519235409796238, -0.0031913125421851873, 0.000517558422870934, -0.02944565750658512, 0.03546567261219025]
Total dimensi: 4096


<b>Adding Vector Databases to ChromaDB</b><br>
We will using `OllamaEmbeddings` with model: `qwen3-embedding:8b`<br>(open-source Embedding model that has achieved state-of-the-art performance across a wide range of downstream application evaluations. The 8B size embedding model ranks No.1 in the MTEB multilingual leaderboard (as of June 5, 2025, score 70.58) and supports 100+ languages. <a href="https://ollama.com/library/qwen3-embedding">[source]</a>)
<br><br>
Then there is two choices for storing Embedding Vectors, either storing it local via `ChromaDB` Vector Database, or load it with `FAISS-gpu` for faster computationality.

In [ ]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="qwen3-embedding:8b",show_progress=True),
    collection_name="mbg-vectordb",
    persist_directory="./rag-mbg-vectordb"
)

/tmp/ipython-input-3356752140.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="qwen3-embedding:8b",show_progress=True),
OllamaEmbeddings: 100%|██████████| 52/52 [00:22<00:00,  2.28it/s]


Using `FAISS` with CUDA-gpu support improve computationality of Vector operations.

In [ ]:
# Build vectorstore
vectorstore = FAISS.from_documents(documents=chunks,
                                   embedding=OllamaEmbeddings(model="qwen3-embedding:8b", show_progress=True))
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

/tmp/ipython-input-2628937837.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="qwen3-embedding:8b", show_progress=True))
OllamaEmbeddings: 100%|██████████| 76/76 [00:29<00:00,  2.56it/s]


### 3. Model Selection
Load the LLM Chat model, we will be using the Qwen3-8B: open-source LLM that improved from Qwen2.5 and very competitive with other LLMs such as Gemma3 and DeepSeek-R1 8B.<br>
For detailed bechmarks and comparisons, here: https://blogs.novita.ai/qwen-3-and-qwen-2-5/

Why use open-source LLMs? Well, there's several reasons:
- Deployment can be local and anywhere
- No need to pay or subscription services
- Privacy matters, as open-source LLMs runs on local, privacy stays on local machine
- Open-source models has worldwide contributions unlike closed-source is classified contributions
- Open-source has massive support of community developers like HuggingFace, llama.cpp, Ollama, Langchain, etc.

In [ ]:
llm = ChatOllama(model='qwen3:8b')

/tmp/ipython-input-884170594.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model='qwen3:8b')


Define the Query Prompt Template

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

<b>Still about Retrieval Strategy</b>

We will be using `MultiQueryRetriever` for main strategy, as it can send multiple queries at one time and one prompt.

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_retriever,
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

Create the Retrieval Chains

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

List to collect retrieved data

In [ ]:
contexts = []
answers = []

Regex string parser to ignore `<think>`...`</think>` blocks by reasoning Qwen3-8B model

In [ ]:
import re

pattern = re.compile(r"<think\b[^>]*>.*?</think>", flags=re.IGNORECASE | re.DOTALL)

# remove <think>...</think> with: pattern.sub("", text)

### 4. Evaluation Dataset Creation
Develop an evaluation dataset that reflects realistic queries researchers might have. Include both simple and complex queries (basic questions, detailed informations, analytical questions, fact checking) that test the system's ability to retrieve and generate accurate information. Around 30 questions should be enough. The ground-truth answer were generated by LLM (ChatGPT) from input PDF MBG Paper.

In [ ]:
qa_pairs = [
    {
        "question": "Apa tujuan utama penelitian tentang Program Makan Bergizi Gratis (MBG) dalam jurnal ini?",
        "answer": "Penelitian ini bertujuan untuk menganalisis perlindungan hukum terhadap Program Makan Bergizi Gratis (MBG) dari perspektif hukum masyarakat di Indonesia. Kajian difokuskan pada dasar hukum, mekanisme pengawasan, partisipasi publik, serta tantangan implementasi agar program berjalan akuntabel, transparan, dan berkeadilan. Penulis menegaskan bahwa MBG harus menjadi instrumen keadilan sosial yang ditopang kerangka hukum kuat dan partisipasi masyarakat aktif untuk menjamin perlindungan anak dan pemenuhan hak dasar. Jawaban: Mengkaji dasar hukum, mekanisme pengawasan, partisipasi publik, serta tantangan implementasi agar program berjalan akuntabel, transparan, dan berkeadilan."
    },
    {
        "question": "Pasal berapa dalam UUD 1945 yang dijadikan dasar hukum untuk menjamin hak atas pangan dan jaminan sosial?",
        "answer": "Berdasarkan hasil analisis dalam jurnal, dasar hukum pemenuhan hak atas pangan dan jaminan sosial terdapat pada Pasal 28C dan Pasal 28H UUD 1945. Pasal 28C menegaskan hak warga negara atas pengakuan, jaminan, perlindungan, dan kepastian hukum yang adil, sedangkan Pasal 28H menyatakan bahwa setiap orang berhak hidup sejahtera, bertempat tinggal, serta memperoleh lingkungan hidup yang baik dan sehat. Kedua pasal ini menjadi fondasi konstitusional bagi program MBG. Jawaban: Pasal 28C dan Pasal 28H UUD 1945."
    },
    {
        "question": "Siapa presiden yang meluncurkan Program MBG dan pada tahun berapa?",
        "answer": "Program Makan Bergizi Gratis (MBG) diluncurkan oleh Presiden Prabowo Subianto pada tahun 2024 sebagai salah satu program unggulan masa awal pemerintahannya. Program ini merupakan bagian dari visi pembangunan manusia unggul dan pemerataan gizi nasional. Jawaban: Presiden Prabowo Subianto pada tahun 2024."
    },
    {
        "question": "Kasus apa yang sering muncul sejak awal pelaksanaan MBG?",
        "answer": "Sejak awal pelaksanaan MBG, berbagai daerah mengalami kasus keracunan massal akibat konsumsi makanan yang tidak memenuhi standar keamanan pangan. Insiden terjadi di wilayah seperti Cianjur, Sukoharjo, dan daerah lain, menunjukkan kelalaian dalam pengawasan, distribusi, dan pengadaan bahan makanan. Jawaban: Kasus keracunan massal akibat konsumsi makanan dari program tersebut."
    },
    {
        "question": "Negara mana yang dijadikan rujukan model MBG oleh pemerintah Indonesia?",
        "answer": "Pemerintah Indonesia menjadikan Jepang sebagai rujukan utama pelaksanaan MBG. Jepang memiliki sistem makan siang sekolah bernama Kyūshoku yang sudah berlangsung sejak pasca-Perang Dunia II, dengan keberhasilan karena adanya pendidikan gizi, kontrol mutu ketat, dan budaya disiplin di sekolah. Jawaban: Jepang dengan sistem Kyūshoku."
    },
    {
        "question": "Mengapa replikasi model Jepang dalam MBG dinilai bermasalah di Indonesia?",
        "answer": "Replikasi model Jepang dalam MBG dinilai bermasalah karena tidak memperhitungkan konteks sosial, ekonomi, dan kelembagaan Indonesia. Kesiapan infrastruktur, sanitasi, serta pengawasan pangan di Indonesia masih lemah, sementara Jepang memiliki sistem pendidikan dan kesehatan yang jauh lebih mapan. Akibatnya, model tersebut sulit diadaptasi secara efektif. Jawaban: Karena kurangnya kesiapan infrastruktur, pengawasan, dan pemahaman konteks sosial-ekonomi Indonesia."
    },
    {
        "question": "Apa saja faktor penyebab utama keracunan makanan dalam program MBG menurut jurnal?",
        "answer": "Jurnal menjelaskan bahwa penyebab utama keracunan makanan berasal dari kelalaian dalam pengadaan bahan, penyimpanan tidak higienis, distribusi yang tidak memenuhi standar, serta lemahnya pengawasan di sekolah. Banyak penyedia katering tidak memiliki fasilitas memadai untuk menjaga keamanan pangan. Jawaban: Kelalaian dalam pengadaan bahan makanan, distribusi, kurangnya pengawasan, dan fasilitas penyimpanan yang tidak memadai."
    },
    {
        "question": "Apa yang dimaksud dengan moral hazard dalam konteks MBG?",
        "answer": "Dalam konteks MBG, moral hazard muncul ketika penyedia katering atau pelaksana program mengejar keuntungan finansial dengan mengabaikan standar keamanan pangan. Fenomena ini diperparah oleh lemahnya pengawasan dan kurangnya partisipasi publik dalam kontrol mutu program. Jawaban: Penyedia katering mengejar keuntungan dengan mengabaikan standar keamanan pangan."
    },
    {
        "question": "Mengapa tanggung jawab hukum dalam kasus keracunan MBG sering kabur?",
        "answer": "Tanggung jawab hukum kabur karena belum ada regulasi khusus yang mengatur pembagian peran antara penyedia makanan, sekolah, dan pemerintah daerah. Ketika terjadi insiden, tidak jelas siapa yang bertanggung jawab secara hukum atas kelalaian dalam distribusi atau pengawasan. Jawaban: Karena tidak adanya regulasi khusus, sehingga tidak jelas apakah tanggung jawab ada pada penyedia makanan, kepala sekolah, atau pemerintah daerah."
    },
    {
        "question": "Apa dasar hukum yang mengatur hak anak sebagai konsumen di Indonesia?",
        "answer": "Hak anak sebagai konsumen dilindungi oleh Undang-Undang Nomor 8 Tahun 1999 tentang Perlindungan Konsumen. UU ini menjamin hak anak untuk mendapatkan barang dan jasa yang aman, termasuk makanan sekolah, serta melarang penyedia memproduksi makanan yang tidak memenuhi standar keamanan pangan. Jawaban: Undang-Undang No. 8 Tahun 1999 tentang Perlindungan Konsumen."
    },
    {
        "question": "Bagaimana bentuk tanggung jawab negara menurut Pasal 111 ayat (2) UU No. 36 Tahun 2009 tentang Kesehatan?",
        "answer": "Menurut Pasal 111 ayat (2) UU No. 36 Tahun 2009, pemerintah bertanggung jawab atas ketersediaan pangan yang aman, bergizi, bermutu, dan tidak membahayakan kesehatan masyarakat. Dalam konteks MBG, hal ini berarti negara wajib memastikan seluruh rantai penyediaan makanan memenuhi standar kesehatan. Jawaban: Negara bertanggung jawab atas ketersediaan pangan yang aman, bermutu, bergizi, dan tidak membahayakan kesehatan masyarakat."
    },
    {
        "question": "Apa contoh negara berkembang yang berhasil mengintegrasikan program makan siang gratis ke dalam sistem pendidikan?",
        "answer": "Brasil menjadi contoh sukses melalui Program Nacional de Alimentação Escolar (PNAE). Program ini mengintegrasikan penyediaan makanan bergizi dengan sistem pendidikan dan melibatkan pemerintah daerah serta petani lokal sebagai pemasok utama, menciptakan sinergi antara gizi, pendidikan, dan ekonomi lokal. Jawaban: Brasil melalui Program Nacional de Alimentação Escolar (PNAE)."
    },
    {
        "question": "Apa masalah utama yang muncul dalam Midday Meal Scheme di India?",
        "answer": "Masalah utama dalam program Midday Meal Scheme di India adalah banyaknya kasus keracunan massal karena bahan makanan berkualitas buruk, lemahnya pengawasan, serta keterlibatan mafia pangan. Hal ini menunjukkan bahwa kebijakan sosial tanpa pengawasan efektif rentan gagal. Jawaban: Kasus keracunan massal akibat kualitas bahan makanan yang buruk, lemahnya pengawasan, dan praktik mafia pangan."
    },
    {
        "question": "Berapa besar anggaran yang dialokasikan pemerintah Indonesia untuk program Makan Siang Gratis?",
        "answer": "Pemerintah Indonesia mengalokasikan sekitar Rp71 triliun untuk program MBG, dengan sasaran 82,9 juta penerima di seluruh jenjang pendidikan dasar dan menengah. Anggaran besar ini menunjukkan skala nasional program, namun juga menimbulkan risiko penyimpangan jika tidak diawasi ketat. Jawaban: Rp71 triliun untuk 82,9 juta penerima."
    },
    {
        "question": "Apa kritik utama terhadap MBG terkait anggaran pendidikan?",
        "answer": "Kritik utama terhadap MBG adalah adanya pengalihan dana Bantuan Operasional Sekolah (BOS) untuk pembiayaan makan siang, sehingga kegiatan belajar-mengajar terganggu. Beberapa laporan juga menunjukkan munculnya kembali praktik pungutan liar di sekolah akibat kekurangan dana operasional. Jawaban: Dana BOS dialihkan untuk program makan siang, sehingga kegiatan belajar-mengajar terganggu dan pungutan liar kembali muncul."
    },
    {
        "question": "Apa prinsip utama dalam pengadaan barang/jasa pemerintah menurut UU No. 2 Tahun 2017?",
        "answer": "UU No. 2 Tahun 2017 menegaskan bahwa pengadaan barang/jasa pemerintah harus berlandaskan pada prinsip transparansi, efisiensi, dan akuntabilitas. Dalam konteks MBG, prinsip ini penting untuk mencegah praktik korupsi dan kolusi dalam pengadaan makanan. Jawaban: Transparansi, efisiensi, dan akuntabilitas."
    },
    {
        "question": "Mengapa partisipasi publik penting dalam pelaksanaan MBG?",
        "answer": "Partisipasi publik penting agar masyarakat, orang tua, dan sekolah dapat terlibat dalam perencanaan dan pengawasan MBG. Pelibatan publik meningkatkan legitimasi, mencegah penyalahgunaan, dan memastikan program sesuai kebutuhan lokal serta budaya pangan setempat. Jawaban: Untuk memastikan program sesuai kebutuhan lokal, meningkatkan kontrol sosial, dan mencegah penyalahgunaan."
    },
    {
        "question": "Apa risiko terbesar jika MBG tidak dikelola dengan transparan?",
        "answer": "Tanpa transparansi, program MBG berisiko tinggi mengalami praktik korupsi, kolusi, dan nepotisme. Ketertutupan informasi juga menurunkan kepercayaan publik terhadap pemerintah dan berpotensi menimbulkan pembangkangan sosial. Jawaban: Korupsi, kolusi, nepotisme, serta menurunnya kepercayaan publik terhadap pemerintah."
    },
    {
        "question": "Apa bentuk perlindungan hukum yang dibutuhkan peserta didik dalam program MBG?",
        "answer": "Peserta didik membutuhkan perlindungan hukum berupa standar higienitas makanan, inspeksi rutin, mekanisme pengaduan dan tanggap darurat, serta kompensasi yang adil bagi korban keracunan. Hal ini sejalan dengan prinsip perlindungan konsumen dan anak dalam hukum nasional. Jawaban: Standar higienitas makanan, inspeksi berkala, mekanisme tanggap darurat, dan ganti rugi yang adil bagi korban keracunan."
    },
    {
        "question": "Apa kesimpulan utama jurnal terkait keberhasilan MBG?",
        "answer": "Jurnal menyimpulkan bahwa keberhasilan MBG hanya dapat dicapai jika program didukung oleh kerangka hukum kuat, sistem pengawasan transparan, partisipasi masyarakat aktif, serta perlindungan konsumen dan anak yang efektif. Tanpa itu, MBG rentan menjadi proyek politis semata. Jawaban: Keberhasilan MBG bergantung pada kerangka hukum yang kuat, transparansi pengadaan, partisipasi masyarakat, serta perlindungan konsumen dan anak."
    },
    {
        "question": "Apa metode penelitian yang digunakan dalam kajian jurnal ini?",
        "answer": "Penelitian ini menggunakan metode hukum normatif dengan pendekatan studi kepustakaan. Analisis difokuskan pada peraturan perundang-undangan, doktrin hukum, dan literatur akademik yang relevan untuk menjelaskan tanggung jawab negara atas program MBG. Jawaban: Metode penelitian hukum normatif dengan studi kepustakaan dan analisis regulasi, doktrin hukum, serta bahan hukum tertulis."
    },
    {
        "question": "Mengapa MBG dianggap berpotensi menjadi proyek populis?",
        "answer": "MBG dianggap berpotensi menjadi proyek populis karena diluncurkan secara cepat dengan motif politik, tanpa kesiapan teknis dan infrastruktur yang memadai. Program lebih difokuskan pada citra politik ketimbang keberlanjutan jangka panjang. Jawaban: Karena pelaksanaannya lebih menekankan citra politik dibanding kesiapan teknis, sehingga berisiko tidak berkelanjutan."
    },
    {
        "question": "Apa kewajiban negara menurut Konvensi Hak Anak terkait program gizi?",
        "answer": "Menurut Konvensi Hak Anak yang diratifikasi melalui Keppres No. 36 Tahun 1990, negara wajib menjamin akses anak terhadap pelayanan kesehatan dan gizi yang optimal. Pelanggaran terhadap kewajiban ini dianggap pengingkaran terhadap komitmen internasional. Jawaban: Negara wajib menjamin akses anak terhadap pelayanan kesehatan dan gizi yang optimal."
    },
    {
        "question": "Apa masalah sanitasi yang memperbesar risiko keracunan dalam MBG?",
        "answer": "Masalah sanitasi seperti kurangnya akses air bersih, fasilitas cuci tangan, dan toilet layak di sekolah menyebabkan meningkatnya risiko kontaminasi makanan. Kondisi ini menurunkan efektivitas program MBG dan meningkatkan potensi keracunan massal. Jawaban: Kurangnya akses air bersih, fasilitas cuci tangan, dan toilet layak di sekolah."
    },
    {
        "question": "Apa peran BPKP, KPK, dan Ombudsman dalam pelaksanaan MBG?",
        "answer": "BPKP, KPK, dan Ombudsman berperan sebagai lembaga pengawas eksternal yang memastikan transparansi, mencegah korupsi, dan menjaga akuntabilitas program. Pengawasan mereka penting untuk mencegah penyimpangan dana publik dan menjaga kredibilitas pelaksanaan MBG. Jawaban: Sebagai lembaga pengawas untuk memastikan transparansi, mencegah korupsi, dan menjaga akuntabilitas program."
    },
    {
        "question": "Bagaimana Jepang menjamin keberhasilan program makan siang sekolahnya?",
        "answer": "Jepang menjamin keberhasilan Kyūshoku dengan membangun infrastruktur sanitasi yang baik, pelatihan tenaga dapur, pendidikan gizi di sekolah, dan sistem kontrol mutu yang ketat. Program ini juga menanamkan nilai disiplin dan tanggung jawab pada siswa. Jawaban: Dengan membangun infrastruktur sanitasi, pelatihan tenaga, pendidikan gizi, dan kontrol mutu yang ketat."
    },
    {
        "question": "Apa risiko hukum bagi penyedia katering jika terbukti lalai dalam MBG?",
        "answer": "Penyedia katering yang lalai dapat dikenai sanksi pidana dan perdata berdasarkan UU Perlindungan Konsumen dan peraturan BPOM, karena melanggar standar keamanan pangan dan menimbulkan kerugian bagi peserta didik. Jawaban: Dapat diproses pidana dan perdata karena melanggar standar keamanan pangan dan merugikan konsumen."
    },
    {
        "question": "Mengapa keterlibatan masyarakat lokal penting dalam pengadaan makanan sekolah?",
        "answer": "Keterlibatan masyarakat lokal memungkinkan menu disesuaikan dengan budaya pangan setempat, menekan biaya logistik, dan memberdayakan petani kecil sebagai pemasok. Pendekatan partisipatif ini terbukti efektif seperti dalam model Brasil (PNAE). Jawaban: Agar menu sesuai budaya pangan lokal, menekan harga, dan memberdayakan petani kecil sebagai pemasok."
    },
    {
        "question": "Apa ancaman terbesar jika negara tidak transparan soal insiden keracunan MBG?",
        "answer": "Ketertutupan informasi mengenai insiden keracunan dapat menimbulkan krisis kepercayaan publik dan memicu resistensi sosial terhadap program pemerintah. Transparansi diperlukan untuk menjaga legitimasi negara. Jawaban: Terjadi krisis kepercayaan masyarakat terhadap pemerintah dan potensi pembangkangan sosial."
    },
    {
        "question": "Apa prinsip rule of law yang harus ditegakkan dalam MBG?",
        "answer": "Prinsip rule of law dalam MBG menekankan pentingnya aturan hukum yang jelas, konsisten, dan ditegakkan tanpa diskriminasi. Program harus berpihak pada keadilan sosial dengan mekanisme sanksi yang tegas agar tidak ada pelanggaran hukum yang dibiarkan. Jawaban: Aturan hukum yang jelas, konsisten, ditegakkan tanpa diskriminasi, serta berpihak pada keadilan sosial."
    }
]


<b>Preparing functions to collect (retrieve) RAG answers from 30 QnA dataset creation</b><br>
The function will iterate every `qa_pairs` and do `retriever.invoke(question)`

In [ ]:
import re

def process_qa_pairs_with_rag(qa_pairs, retriever, chain):
    contexts = []
    answers = []

    for i, qa_pair in enumerate(qa_pairs):
        content = []
        retrieval = retriever.invoke(qa_pair["question"])

        for item in retrieval:
            content.append(item.page_content)
        contexts.append(content)

        # Generate answer
        answer = chain.invoke(qa_pair["question"])
        answer = pattern.sub("", answer)  # remove <think>...</think>
        if answer.startswith("\n\n"):
            answer = answer.replace("\n\n", "")
        answers.append(answer)

        print(f"Question {i + 1}: {qa_pair['question']}")
        print(f"Generated Answer {i + 1}: {answer}")
        print(f"Original Answer {i + 1}: {qa_pair['answer']}\n")

    return contexts, answers

Let\'s execute the retriever loop

In [ ]:
contexts, answers = process_qa_pairs_with_rag(qa_pairs, retriever, chain)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.44it/s]


Question 1: Apa tujuan utama penelitian tentang Program Makan Bergizi Gratis (MBG) dalam jurnal ini?
Generated Answer 1: Tujuan utama penelitian tentang Program Makan Bergizi Gratis (MBG) dalam jurnal ini adalah **menganalisis kelalaian sistemik, inkonsistensi kebijakan, dan lemahnya tata kelola dalam pelaksanaan program tersebut**, serta mengevaluasi efektivitas mekanisme kontrol mutu makanan. Penelitian ini juga mengeksplorasi dampak etis dan politik dari program, termasuk risiko penggunaan MBG sebagai alat populisme elektoral, serta mengkritik kurangnya standar pengawasan pangan yang menyebabkan insiden keracunan massal. Pendekatan normatif digunakan untuk menilai kerangka hukum dan kebijakan yang seharusnya mengatur program ini, dengan tujuan memberikan solusi hukum dan kebijakan untuk memperbaiki keberlanjutan dan integritas MBG.
Original Answer 1: Penelitian ini bertujuan untuk menganalisis perlindungan hukum terhadap Program Makan Bergizi Gratis (MBG) dari perspektif hukum masya

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  6.04it/s]


Question 2: Pasal berapa dalam UUD 1945 yang dijadikan dasar hukum untuk menjamin hak atas pangan dan jaminan sosial?
Generated Answer 2: Pasal 28C dan Pasal 28H dalam UUD 1945 dijadikan dasar hukum untuk menjamin hak atas pangan dan jaminan sosial. Kedua pasal tersebut menyatakan bahwa setiap warga negara berhak atas pengakuan, jaminan, perlindungan, dan kepastian hukum yang adil, termasuk hak atas pangan yang layak dan bergizi, serta perlindungan sosial yang merata.
Original Answer 2: Berdasarkan hasil analisis dalam jurnal, dasar hukum pemenuhan hak atas pangan dan jaminan sosial terdapat pada Pasal 28C dan Pasal 28H UUD 1945. Pasal 28C menegaskan hak warga negara atas pengakuan, jaminan, perlindungan, dan kepastian hukum yang adil, sedangkan Pasal 28H menyatakan bahwa setiap orang berhak hidup sejahtera, bertempat tinggal, serta memperoleh lingkungan hidup yang baik dan sehat. Kedua pasal ini menjadi fondasi konstitusional bagi program MBG. Jawaban: Pasal 28C dan Pasal 28H UUD 1945

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  6.14it/s]


...

Question 28: Mengapa keterlibatan masyarakat lokal penting dalam pengadaan makanan sekolah?
Generated Answer 28: Keterlibatan masyarakat lokal dalam pengadaan makanan sekolah penting untuk beberapa alasan berdasarkan konteks yang diberikan:  1. **Mencegah Korupsi dan Kolusi**: Transparansi serta partisipasi masyarakat menjadi kunci untuk menghindari praktik korupsi, seperti manipulasi tender, mark-up harga, dan kolusi antar pejabat dengan penyedia jasa. Tanpa keterlibatan masyarakat, dana program berisiko disalahgunakan, yang dapat merugikan hak anak dan kepentingan publik.  2. **Memastikan Kesesuaian dengan Kebutuhan Lokal**: Partisipasi masyarakat memungkinkan desain program yang lebih sesuai dengan kebutuhan geografis dan budaya setempat. Standarisasi menu yang tidak mempertimbangkan kebutuhan lokal bisa menciptakan limbah makanan dan tidak efektif dalam memenuhi asupan gizi.  3. **Mendorong Partisipasi Aktif dan Kontrol Sosial**: Keterlibatan warga, seperti orang tua siswa, komite 

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.86it/s]


Question 29: Apa ancaman terbesar jika negara tidak transparan soal insiden keracunan MBG?
Generated Answer 29: Acaman terbesar jika negara tidak transparan soal insiden keracunan MBG adalah **risiko pembangkangan sosial terhadap program negara**. Ketidaktransparansi memperparah kelalaian sistemik dalam pengelolaan program, seperti kegagalan kontrol mutu makanan, koordinasi lintas sektor, dan akuntabilitas. Hal ini mengurangi kepercayaan masyarakat, menciptakan ketidakstabilan citra pemerintah, dan berpotensi memicu ketidakpuasan yang berujung pada penolakan atau pembangkangan terhadap kebijakan nasional. Selain itu, penanganan yang tidak terbuka juga memperkuat kritik internasional dan merusak tata kelola proyek sosial, yang berdampak pada stabilitas diplomatis dan kredibilitas pemerintahan.
Original Answer 29: Ketertutupan informasi mengenai insiden keracunan dapat menimbulkan krisis kepercayaan publik dan memicu resistensi sosial terhadap program pemerintah. Transparansi diperlukan 

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  6.31it/s]


Question 30: Apa prinsip rule of law yang harus ditegakkan dalam MBG?
Generated Answer 30: Prinsip **rule of law** yang harus ditegakkan dalam program MBG (Makan Siang Gratis) adalah bahwa program ini harus beroperasi dalam bingkai aturan hukum yang **jelas, konsisten, dan ditegakkan tanpa diskriminasi** (seperti yang dijelaskan dalam referensi Rawls, 1971). Hal ini berarti:  
1. **Keterbukaan dan kejelasan regulasi**: MBG harus diatur dalam peraturan yang jelas dan berlaku umum, bukan hanya berbasis kebijakan politis.  
2. **Konsistensi dalam penerapan**: Penegakan hukum dan standar operasional harus konsisten, sehingga tidak ada pengecualian atau diskriminasi dalam distribusi makanan atau pengelolaan anggaran.  
3. **Tidak diskriminatif**: Hakikat program MBG sebagai bentuk jaminan sosial harus ditegakkan secara adil, tanpa membeda-bedakan kelompok masyarakat.  Ketidaktegasan dalam hal ini, seperti yang disebutkan dalam penelitian Naafi (2024), berpotensi memicu konflik sosial dan me

### 5. Evaluation
Here we will be using RAGAS evaluation metrics for evaluating RAG results.<br>
There are several RAGAS metrics:<br>
1. Faithfullness: Faithfulness measures how factually consistent the generated answer is with the provided context. It ensures that the claims made in the answer can be directly inferred from the given context. A faithful answer means that all statements in the answer are supported by the context.<br>(Example: If the context states that "Einstein was born on March 14, 1879," an answer saying he was born on March 20, 1879, would have low faithfulness.)<br>
2. Answer Relevancy: This metric assesses how relevant the generated answer is to the original question. It focuses on whether the answer addresses the query directly and avoids unnecessary or unrelated information. Relevance is often measured by generating variations of the question based on the answer and comparing them to the original question using cosine similarity.<br>(Example: For a question asking "What is the capital of France?", an answer stating "Paris is the capital of France" is highly relevant, while an answer discussing French cuisine would score lower.).<br>
3. Context Precision: Context precision evaluates how accurately the retrieved context provides the necessary information to answer the question. It measures whether the relevant information is ranked higher among the retrieved documents, focusing on the signal-to-noise ratio in the retrieval process.<br>(Example: If the context retrieved is directly related to the query without much irrelevant information, the precision is high.).<br>
4. Context Recall: Context recall measures how much of the relevant information from the ground truth is included in the retrieved context. It compares the retrieved context with the expected answer to see if all necessary details are captured.<br>(Example: If the ground truth for a question includes multiple details, and the retrieved context covers most of these, the recall would be high.).<br>
5. Answer Similarity: Answer similarity assesses how close the generated answer is to the ground truth by comparing the semantic similarity between the two. This metric ensures that even if different words are used, the meaning conveyed is similar.<br>(Example: If the ground truth answer is "Paris is the capital of France," and the generated answer is "The capital of France is Paris," the similarity would be high.).<br>

<b>RAGAS Evaluation preparation</b><br>
Before starting RAGAS eval, the RAGAS have input criteria as below:<br>
`{`<br>
`  "question": List[str],`<br>
`  "answer": List[str],`<br>
`  "contexts": List[List[str]],`<br>
`  "reference": List[str]`<br>
`}`<br>

Where it explains:
- "question": is the input question
- "answer" is generated answer from LLM RAG
- "contexts" is context chunks from document chunks
- "reference" is ground truth (actual or references) answers

In [ ]:
# Formatting the dataset for inputting to RAGAS

from datasets import Dataset, DatasetDict

rag_question = []
rag_answer = []
rag_ground_truth = []
rag_context = contexts

for item in qa_pairs:
    rag_question.append(item["question"])
    rag_ground_truth.append(item["answer"])

for item in answers:
    rag_answer.append(item)

d = {
    "question": rag_question,
    "answer": rag_answer,
    "contexts": rag_context,
    "reference": rag_ground_truth
}
dataset = Dataset.from_dict(d)

Then the Dictionary dataset should be converted into HuggingFace Dataset using `Dataset.from_dict()`

In [ ]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 30
})

<b>Preparing RAGAS metrics, LLM Wrapper, and Embedding Wrapper</b><br>
for evaluating with RAGAS, we will use `Qwen2.5:7b` and `Qwen3-embedding:8b` for faster computation evaluation.<br>
Also we setting up longer timeouts to prevent many timeouts while evaluating RAGAS.

In [ ]:
!ollama list

NAME                  ID              SIZE      MODIFIED           
qwen2.5:3b            357c53fb659c    1.9 GB    About a minute ago    
qwen3:8b              500a1f067a9f    5.2 GB    About a minute ago    
qwen3-embedding:4b    df5bd2e3c74c    2.5 GB    About a minute ago    
qwen2.5:7b            845dbda0ea48    4.7 GB    About an hour ago     
qwen3-embedding:8b    64b933495768    4.7 GB    About an hour ago     


In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from ragas.run_config import RunConfig
from ragas.llms.base import LangchainLLMWrapper
from ragas.embeddings.base import LangchainEmbeddingsWrapper

# Increase timeout to 1200 seconds (20 minutes) for longer timeout
run_config = RunConfig(
    max_retries=99999,
    max_wait=600,
    timeout=1200,
    max_workers=1,
    log_tenacity=True
)

wrapped_llm = LangchainLLMWrapper(ChatOllama(model='qwen2.5:7b'), run_config=run_config)
wrapped_embeddings = LangchainEmbeddingsWrapper(OllamaEmbeddings(model='qwen3-embedding:8b'), run_config=run_config)

/tmp/ipython-input-4261840290.py:25: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  wrapped_embeddings = LangchainEmbeddingsWrapper(OllamaEmbeddings(model='qwen3-embedding:8b'), run_config=run_config)


<b>Execute the RAGAS eval</b>

In [ ]:
result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        answer_similarity,
    ],
    llm=wrapped_llm,
    embeddings=wrapped_embeddings
)

df_ragas = result.to_pandas()

Evaluating:   0%|          | 0/150 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[0]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[2]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[7]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[10]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[12]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[15]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[17]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[20]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[22]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[25]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[27]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[30]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[32]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[35]: TimeoutError()
ERROR:ragas.executor:Exceptio

<b>Viewing RAGAS eval result in Pandas DataFrame format</b>

In [ ]:
df_ragas

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,answer_similarity
0,Apa tujuan utama penelitian tentang Program Ma...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Tujuan utama penelitian tentang Program Makan ...,Penelitian ini bertujuan untuk menganalisis pe...,NaN,1.000000,NaN,0.878864,0.794339
1,Pasal berapa dalam UUD 1945 yang dijadikan das...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Pasal 28C dan Pasal 28H dalam UUD 1945 dijadik...,"Berdasarkan hasil analisis dalam jurnal, dasar...",NaN,0.000000,NaN,0.772772,0.827427
2,Siapa presiden yang meluncurkan Program MBG da...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Presiden yang meluncurkan Program Makan Bergiz...,Program Makan Bergizi Gratis (MBG) diluncurkan...,NaN,1.000000,NaN,0.740310,0.924215
3,Kasus apa yang sering muncul sejak awal pelaks...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Kasus yang sering muncul sejak awal pelaksanaa...,"Sejak awal pelaksanaan MBG, berbagai daerah me...",NaN,0.666667,NaN,0.638595,0.863598
4,Negara mana yang dijadikan rujukan model MBG o...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Negara yang dijadikan rujukan model MBG oleh p...,Pemerintah Indonesia menjadikan Jepang sebagai...,NaN,1.000000,NaN,0.867576,0.878364
5,Mengapa replikasi model Jepang dalam MBG dinil...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Replikasi model Jepang dalam MBG dinilai berma...,Replikasi model Jepang dalam MBG dinilai berma...,NaN,1.000000,NaN,0.812232,0.905232
6,Apa saja faktor penyebab utama keracunan makan...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,"Berdasarkan jurnal tersebut, faktor penyebab u...",Jurnal menjelaskan bahwa penyebab utama keracu...,NaN,0.500000,NaN,0.870091,0.680111
7,Apa yang dimaksud dengan moral hazard dalam ko...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,"Dalam konteks MBG (Bantuan Pangan Sekolah), **...","Dalam konteks MBG, moral hazard muncul ketika ...",NaN,0.666667,NaN,0.759887,0.913032
8,Mengapa tanggung jawab hukum dalam kasus kerac...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Tanggung jawab hukum dalam kasus keracunan MBG...,Tanggung jawab hukum kabur karena belum ada re...,NaN,1.000000,NaN,0.821422,0.764162
9,Apa dasar hukum yang mengatur hak anak sebagai...,[1330 |Jurnal Penelitian Ilmiah Multidisipline...,Dasar hukum yang mengatur hak anak sebagai kon...,Hak anak sebagai konsumen dilindungi oleh Unda...,NaN,1.000000,NaN,0.765611,0.800007


The evaluation scores can be exported to CSV format if decided

In [ ]:
df_ragas.to_csv("EvaluationScores.csv", encoding="utf-8", index=False)

### 6. Recommendation
The RAG evaluation using the RAGAS library successfully produced results for three out of five evaluation metrics `context recall`, `answer relevancy`, and `answer similarity` showing overall solid retrieval and response quality, with most scores ranging between 0.7 and 0.9. However, the `context precision` and `faithfulness` metrics failed due to TimeoutError exceptions, preventing full evaluation of retrieval accuracy and factual consistency. It is also noteworthy that the `context recall` values are sharply polarized (mostly 1.0 or 0.667), likely influenced by the fact that the _ground-truth references_ were generated by an LLM (ChatGPT) rather than human annotations, which can cause the recall metric to overestimate or underestimate retrieval quality due to stylistic or phrasing variations. Despite these constraints, the available metrics suggest that the RAG pipeline performs well in maintaining contextual relevance, but further refinement and stability improvements are needed for more balanced and comprehensive evaluations.